In [62]:
import numpy as np
from copy import copy
import tensorflow as tf
from importlib import reload
from random import randint

import environment as env
import agent
import agents.random_agent as ra
import agents.q_no_hidden as qnh

In [76]:
reload(env)
reload(agent)
reload(ra)
reload(qnh)

<module 'agents.q_no_hidden' from 'C:\\Users\\abeol\\Git\\Deep-Notakto\\deepnotakto\\agents\\q_no_hidden.py'>

In [77]:
e = env.Env(3)
a1 = qnh.QNoHidden(3)
a2 = ra.RandomAgent()
e.play_cvc(a1, a2, True)

Turn #0

X O O 
O O O 
O O O 

Turn #1

X X O 
O O O 
O O O 

Turn #2

X X O 
X O O 
O O O 

Turn #3

X X X 
X O O 
O O O 

Player 1 Wins!


In [78]:
env.Env(4).play_hvh()


O O O O 
O O O O 
O O O O 
O O O O 

Player 1
Next Piece: exit


False